# **Ejercicio 3: Modelos de Regresión**

**Modelos de Regresión.** Considere el conjunto de datos **Wind Speed**. Implemente la versión de regresión de cada uno de los modelos estudiados en clases, para predecir la velocidad del viento horaria (**VENTO, VELOCIDADE HORARIA (m/s)**) en el conjunto de datos suministrado. Construya una tabla de error con las métricas usuales de regresión: **MAPE, RMSE, R2** (ver **Table 2**). Además, agregue pruebas de independencia y normalidad para los residuos: **Ljung-Box p-value** y **Jarque-Bera p-value**. Realice particiones de entrenamiento, validación y prueba, basándose en lo descrito en la **Figura 2**. Estas particiones deben seguir la tendencia de la velocidad del viento. Genere una figura que represente la velocidad del viento y su predicción. Utilice las métricas **MAPE, RMSE, R2** en la fase de validación para seleccionar la mejor predicción. Identifique cuál de las métricas es la más adecuada. Use también la optimización bayesiana para seleccionar los mejores hiperparámetros del modelo de regresión óptimo. 

El pliegue de validación en cada partición debe estar siempre ubicado en el porcentaje final de cada partición, ya que el tiempo es fundamental en dichas predicciones. No tiene sentido predecir el pasado conocido con el futuro. Entre los períodos diarios **T = 7, 14, 21, 28**, indique cuál corresponde a la mejor ventana de predicción para el entrenamiento. Tenga en cuenta que **TimeSeriesSplit** no aplica en este problema, por lo que no debe utilizarla, dado que dichos pliegues no corresponden a los solicitados en este ejercicio. Defina una función para construir los pliegues para este ejercicio.

| Modelo                 | MAPE | RMSE | R²   | Ljung-Box p-value | Jarque-Bera p-value |
|-----------------------|------|------|------|--------------------|----------------------|
| K-NN                  | ...  | ...  | ...  | ...                | ...                  |
| Linear Regression      | ...  | ...  | ...  | ...                | ...                  |
| Ridge Regression       | ...  | ...  | ...  | ...                | ...                  |
| Lasso Regression       | ...  | ...  | ...  | ...                | ...                  |

**Cuadro 2:** Modelo de regresión para **mean sale price**.


## **Librerías y modulos necesarios**

In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,make_scorer, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# **Modelo de regresión: Wind Speed**

## **Métricas** 

### **MAPE: Mean Absolute Percentage Error**

In [ ]:
def MAPE(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

### **RMSE: Root Mean Square Error**


In [ ]:
def RMSE(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred)),

### **R2: Coefficient of Determination**

In [ ]:
def R2(y_true, y_pred):
    return r2_score(y_true, y_pred)

## **Datos**

Después de haber realizado un análisis descriptivo de la base de datos original (detallado [Wind Speed - Ejercicio 1](https://kmarcela11.github.io/Parcial2_MachineLearning/Ejercicio1.html#wind-speed)
), finalmente se ha obtenido una base de datos preparada para realizar el modelo, la cual ha sido almacenada en un archivo CSV.

### **Diferencias entre la base de datos original y la del modelo**

- La nueva base de datos presenta nombres de columnas más legibles y mejor organizados.
- Se han eliminado variables poco significativas para el modelo, como:
  - La variable de **hora**.
- Se han excluido variables altamente correlacionadas, tales como:
  - `hum_horaria`
  - `pres_atmo`


In [6]:
data = pd.read_csv('C:\\Users\\kamac\\OneDrive\\Desktop\\MachineLearningUN\\data_modelo.csv')

In [7]:
data

,dir_viento,vel_viento,hum_max,hum_min,temp_max,temp_min,precip_total,rafaga_max,pres_max,pres_min
0,0.809017,1.8,69.0,60.0,22.6,20.7,0.0,3.8,888.2,887.7
1,0.965926,2.7,62.0,55.0,24.2,22.5,0.0,4.7,888.4,888.2
2,0.891007,2.0,56.0,50.0,25.5,24.3,0.0,4.9,888.4,888.1
3,0.848048,2.5,52.0,44.0,27.4,25.0,0.0,5.8,888.1,887.4
4,0.224951,2.4,50.0,43.0,27.1,25.5,0.0,5.8,887.4,886.5
...,...,...,...,...,...,...,...,...,...,...
87688,-0.615661,5.6,83.0,78.0,21.8,21.1,0.0,12.3,879.8,879.1
87689,-0.469472,4.9,84.0,79.0,21.7,21.0,0.0,9.9,879.2,878.9
87690,-0.484810,4.5,86.0,82.0,21.2,20.6,0.0,8.9,879.8,879.2
87691,-0.484810,3.2,88.0,85.0,20.6,20.2,0.0,8.0,880.5,879.6


Nótese que la base de datos actual cuenta con **87693** observaciones y **10** columnas

Ahora, se asignan las variables predictoras y la variable objetivo, que en nuestro caso es la velocidad del viento (`vel_viento`).

In [9]:
X = data.drop(columns = ['vel_viento'])
y = data['vel_viento']

Ahora, usaremos un diccionario para almacenar todos los modelos que se van a entrenar.

In [14]:
modelos = {
    'K-NN Regressor': KNeighborsRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso()
}